### 학습의 시작부터 끝까지
##### 파라미터 초기화 -> 예측 계산 -> 손실 계산 -> 그레디언트 계산 -> 가중치 한 단계 갱신 -> 과정 반복 -> 학습 종료
## 4.4.4 경사 하강법 요약
##### .....................┎> [손실 측정] -> [그레디언트 계산] ┓
##### [초기화] -> [예측] <-------------------------- [가중치 한 단계 갱신] -> [훈련 종료]
##### 시작단계(파라미터 초기화) 에서는 (1) 모델의 가중치를 임의의 값으로 설정하거나(밑바탁부터 학습), 사전에 학습된 모델로부터 설정(전이 학습)할 수 있다.
##### 손실 계산 -> '손실 함수'로 모델의 출력과 목표 타깃값을 비교한다.
##### 그레디언트 -> 미적분으로 계산, 파이토치는 자동으로 계산해준다. 또한, 경사진 정도에 '학습률'을 곱해서 '한 번에 움직여야 하는 양'을 알 수 있다.
##### 과정 반복 -> 가장 낮은 지점에 도달할 때까지 반복, 도달시 멈춘다.

## 4.5 MNIST 손실함수
#### 독립변수 x들은 이미 준비되었다. 따라서, 이를 단일 텐서로 엮어 행렬의 목록(랭크3 텐서)을 만들고, 다시 벡터의 목록(랭크2 텐서)로 바꿔준다. 이 작업에는 view 메소드를 이용한다.
##### * 파이토치의 view(): 담긴 데이터는 건드리지 않고 텐서의 모양만 바꿔줌.

In [1]:
from fastai.vision.all import *

path= untar_data(URLs.MNIST_SAMPLE)
threes= (path/'train'/'3').ls().sorted()
sevens= (path/'train'/'7').ls().sorted()

three_tensors= [tensor(Image.open(o)) for o in threes]
seven_tensors= [tensor(Image.open(o)) for o in sevens]

stacked_threes= torch.stack(three_tensors).float()/255
stacked_sevens= torch.stack(seven_tensors).float()/255

train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28 * 28)

### torch.cat([stacked_threes, stacked_sevens]).view(-1, 28 * 28)
##### * torch.cat(): 원하는 dimension 방향으로 텐서를 나란하게 쌓는다. 기본 dimension 값은 0 이다.
##### ** view(): 텐서에 담긴 데이터는 건드리지 않고, 텐서 모양만 바꿔준다.
##### *** -1 : 해당 축을 모든 데이터에 들어맞을 만큼 크게 만든다.
##### ****28*28: 28*28 사이즈까지.

### 각 이미지에 레이블이 필요하다.
##### '3' 과 '7' 에는 각각 1과 0을 사용

In [2]:
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
train_x.shape, train_y.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

### tensor.unsqueeze()
##### * squeeze함수의 반대로 1인 차원을 생성하는 함수이다. 그래서 어느 차원에 1인 차원을 생성할 지 꼭 지정해주어야한다.
##### ** tensor.squeeze(): 차원이 1인 차원을 제거해준다. 따로 차원을 설정하지 않으면 1인 차원을 모두 제거한다. 그리고 차원을 설정해주면 그 차원만 제거한다.

### 파이토치의 Dataset은 (x,y) 튜플을 반환하기를 요구한다.

In [4]:
dset = list(zip(train_x, train_y))
x, y = dset[0]
x.shape, y

(torch.Size([784]), tensor([1]))

### zip(list1, list2)
##### 리스트의 인덱스끼리 묶어 '튜플'로 구성하는 함수

In [6]:
valid_3_tens= torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_7_tens= torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])

valid_3_tens= valid_3_tens.float()/255
valid_7_tens= valid_7_tens.float()/255

valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x, valid_y))

### 이번에는 각 픽셀에 임의로 초기화된 가중치가 필요하다(7단계중 초기화 단계).

In [7]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

weights = init_params((28*28, 1))

### (torch.randn(size)*std).requires_grad_()
##### randn(size): 정수 랜덤값으로 초기화
##### requires_grad_(): 미분 필요 태그 삽입
### weights * pixels 형식의 함수는 충분히 유연하지 않다.
##### (픽셀값이 0이라면 곱한 결과는 항상 0이 되기 때문; 직선 방정식 'y=w*x+b'에서 b(bias)를 기억하라.)
##### b 또한 임의의 숫자로 초기화

In [8]:
bias = init_params(1)

### y = w*x + b
##### w: 가중치(weight)
##### b: 편향(bias)
##### w 와 b 를 통틀어 파라미터(parameter)라고 한다.
### .
### 이제는 단일 이미지에 대한 예측을 계산할 수 있다.

In [9]:
(train_x[0] * weights.T).sum() + bias

tensor([-1.7580], grad_fn=<AddBackward0>)

##### * 파이썬에서는 @연산자가 행렬 곱셈을 표현.

### 시도해보기

In [10]:
def linear1(xb): return xb@weights + bias
preds = linear1(train_x)
preds

tensor([[ -1.7580],
        [-10.3833],
        [ -3.8600],
        ...,
        [  1.6266],
        [  0.9342],
        [ -1.4529]], grad_fn=<AddBackward0>)

### return xb @ weights + bias
##### * batch @ weights + bias
##### ** 활성화 함수(모든 신경망의 가장 기본인 두 방정식 중 하나.)

### 정확도를 검사해보자.
##### * 3 또는 7인지를 판단하려면, 출력값이 0.5보다 큰지를 검사하면 된다.

In [11]:
corrects= (preds> 0.5).float()== train_y
corrects

tensor([[False],
        [False],
        [False],
        ...,
        [False],
        [False],
        [ True]])

In [12]:
corrects.float().mean().item()

0.47047433257102966

### 가중치 하나를 바꿨을 때, 정확도에서 일어나는 변화 실험

In [14]:
weights[0] *= 1.0001
preds= linear1(train_x)
((preds>0.5).float()== train_y).float().mean().item()

RuntimeError: ignored

### 일반적인 그래디언트(기울기) 함수: (yn-yo) / (xn-xo)
##### 3 -> 7 혹은 7 -> 3 으로 변할 때만 바뀜...
##### 따라서, 작은 변화 => 영향 미미. 거의 항상 0이됨.
##### 그렇다면,
### "약간 '더 나은' 예측"은 어떻게?
##### * 올바른 정답이 3일때, 점수가 '약간' 더 높고, 정답이 7일 때, 점수가 '약간' 더 낮을 때.
##### 따라서,
### 손실 함수는 
##### 이미지가 3인지에 대한 예측으로 0~1 사잇값을 가지는 prds라는 한 인자를 정의할 수 있다.(벡터)
##### 또한, 목적이 예측값과 참값(타겟 또는 레이블) 사이의 '다름'을 측정하는 것이므로, 0또는 1의 값을 가지는 trgts라는 인자를 정의한다.(벡터)

In [15]:
trgts= tensor([1, 0, 1])
prds= tensor([0.9, 0.4, 0.2])

### predictions와 targets 사이의 거리를 측정하는 손실 함수를 만든다.

In [17]:
def mnist_loss(predictions, targets):
  return torch.where(targets==1, 1-predictions, predictions).mean()

torch.where(trgts==1, 1-prds, prds)

tensor([0.1000, 0.4000, 0.8000])

### torch.where(a, b, c) !! 중요 !!
##### * [b[i] if a[i] else c[i] for i in range(len(a))]와 같은 형식의 리스트 컴프리헨션 비슷한 작업을 수행한다.(CUDA와 텐서로 인한 속도 면에서 강점)
##### ** 풀이하면, 정답이 1일때, 예측이 1과 떨어진 정도를,
##### 정답이 0일 때, 예측이 0과 떨어진 정도를 측정하고 난 후,
##### 구한 모든 거리의 평균을 구하는 일을 한다.

In [18]:
mnist_loss(prds, trgts)

tensor(0.4333)

##### * 손실은 스칼라값이어야 하므로, mnist_loss함수는 각 손실의 평균을 반환한다.

In [19]:
mnist_loss(tensor([0.9, 0.4, 0.8]), trgts)

tensor(0.2333)

##### *'거짓 타겟'에 대한 예측을 "0.2 -> 0.8"로 하면, 손실이 줄어들어 더 나은 예측을 나타낸다.

##### 현재 정의된 mnist_loss 함수에는 예측이 항상 0과 1 사잇값이라고 가정하는 문제가 존재.
##### 따라서, 실제로도 값이 0과 1 사이가 되도록 강제해야함. 어떻게 해야하는가?(다음장에)